<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/Group_9_Final%F0%9F%8F%86%F0%9F%9B%92Shop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🛒Shop</font>](https://www.kaggle.com/competitions/26jun23jh-shop/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
#from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23jh-shop    # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23jh-shop
Using competition: 26jun23jh-shop
  teamId  teamName               submissionDate       score    
--------  ---------------------  -------------------  -------  
10637586  Tony Sheller           2023-07-08 15:52:22  0.99324  
10637379  Florian Muellerklein   2023-07-08 00:14:48  0.99300  
10638614  Alayna Stepp           2023-07-06 01:34:40  0.99272  
10631355  Harris Rose            2023-07-08 16:37:58  0.99264  
10631144  Kewei Cai              2023-07-07 05:40:11  0.99228  
10590634  kaggleintro            2023-07-05 01:42:02  0.99008  
10638103  Group 15               2023-07-08 03:16:47  0.98588  
10633911  Matthew Renze          2023-07-07 15:47:37  0.98500  
10644064  Group 2                2023-07-08 08:53:50  0.98156  
10637839  Larry Walker           2023-07-08 00:03:01  0.98124  
10642100  Solomon Gruse          2023-07-07 03:06:27  0.98060  
10651039  Philip-Spencer         

In [ ]:
!pip -q install -U tensorflow_addons > log

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 372 ms, sys: 2.77 ms, total: 375 ms
Wall time: 378 ms


In [ ]:
df = pd.read_csv('XY_Shop.csv'); df

,Adm,AdmDur,Inf,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
0,0,0.00,0,0.0,18,132.99,3.82e-02,5.45e-02,0.0,0.0,4,3,1,1,2,0,1,NaN
1,1,0.00,0,0.0,37,1150.20,1.25e-03,3.03e-02,0.0,0.0,11,2,2,4,2,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499998,0,0.00,0,0.0,27,1185.14,0.00e+00,1.59e-03,0.0,0.0,5,2,2,2,3,0,1,0.0
499999,6,51.36,0,0.0,59,1898.21,0.00e+00,3.22e-03,0.0,0.0,12,2,2,2,1,0,0,0.0


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Adm      500000 non-null  int64  
 1   AdmDur   500000 non-null  float64
 2   Inf      500000 non-null  int64  
 3   InfDur   500000 non-null  float64
 4   Prd      500000 non-null  int64  
 5   PrdDur   500000 non-null  float64
 6   BncRt    500000 non-null  float64
 7   ExtRt    500000 non-null  float64
 8   PgVal    500000 non-null  float64
 9   SpclDay  500000 non-null  float64
 10  Mo       500000 non-null  int64  
 11  OS       500000 non-null  int64  
 12  Bsr      500000 non-null  int64  
 13  Rgn      500000 non-null  int64  
 14  TfcTp    500000 non-null  int64  
 15  VstTp    500000 non-null  int64  
 16  Wkd      500000 non-null  int64  
 17  Rev      450000 non-null  float64
dtypes: float64(8), int64(10)
memory usage: 68.7 MB


In [ ]:
vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
tXY = df.query('Rev==Rev')                     # slice training sample
tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
NumFeatures = list(tX.select_dtypes(include='float').columns)
print('Numeric features: ', NumFeatures)       # numeric/quantitative feature names

Numeric features:  ['AdmDur', 'InfDur', 'PrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay']


In [ ]:
# def ScatterCorrHist(df):
#   def corrdot(*args, **kwargs):
#     # credit: https://stackoverflow.com/questions/48139899
#     corr_r = args[0].corr(args[1], 'pearson')
#     corr_text = f"{corr_r:2.2f}".replace("0.", ".")
#     ax = plt.gca();
#     ax.set_axis_off();
#     msz = abs(corr_r) * 5000   # marker size
#     fsz = abs(corr_r) * 40 + 5 # font size
#     ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
#     ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

#   sns.set(style='white', font_scale=.8);
#   g = sns.PairGrid(df, aspect=1, diag_sharey=False);
#   g.fig.set_size_inches(20,10)
#   g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
#   g.map_diag(sns.histplot, kde_kws={'color':'black'});
#   g.map_upper(corrdot);
#   g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
#   _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
#   _ = plt.show();

# df_ = tXY.loc[(tXY[NumFeatures]>0).all(axis=1), NumFeatures+['Rev']].sample(n=100, random_state=0)
# # df_ = df.select_dtypes(include='float').query('AdmDur>0 and InfDur>0 and PrdDur>0 and BncRt>0 and ExtRt>0 and PgVal>0 and SpclDay>0').sample(n=100, random_state=0)
# ScatterCorrHist(df_)

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

**FYI: This colab is being run on GPU.**

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight
from copy import deepcopy

#df = pd.read_csv('XY_Shop.csv')


# Make New Columns Combining Information that is correlated
def combineFeatures(df):
    '''
    Combine a few features
    '''
    pd.set_option('mode.chained_assignment', None)
    df['AdmAdmDur'] = df.AdmDur/df.Adm
    df['AdmAdmDur'] = df['AdmAdmDur'].fillna(0)

    df['InfInfDur'] = df['InfDur']/df['Inf']
    df['InfInfDur'] = df.InfInfDur.fillna(0)

    df['PrdPrdDur'] = df['PrdDur']/df['Prd']
    df['PrdPrdDur'] = df.PrdPrdDur.fillna(0)
    #df.drop(['Adm','AdmDur','Inf','InfDur','Prd','PrdDur'],axis=1, inplace=True)
    return df

df = combineFeatures(deepcopy(df))



# track which features are numeric and categorical
numerical_feats = ['Adm', 'AdmDur','AdmAdmDur', 'Inf', 'InfDur','InfInfDur', 'PrdDur','PrdPrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay', 'Prd']
categorical_feats = ['OS', 'Bsr', 'Rgn', 'TfcTp', 'VstTp', 'Wkd', 'Mo']


# get one hot encoded featuers without dropping originals
df_dummies = pd.get_dummies(df[categorical_feats], columns=categorical_feats)
df = pd.concat((df, df_dummies), axis=1)

# make the categoricals all 0 indexed
for cat_feat in categorical_feats:
  df[cat_feat] = LabelEncoder().fit_transform(df[cat_feat])

onehot_feats = [col_nm for col_nm in df.columns if '_' in col_nm]

test_data = df.query('Rev!=Rev').drop('Rev', axis=1)                   # slice a test sample
train_data = df.query('Rev==Rev')                                      # slice training sample
train_x, train_y = train_data.drop('Rev', axis=1), train_data.Rev      # split into training I/O

train_x[numerical_feats] = np.log(train_x[numerical_feats] + 1)
test_data[numerical_feats] = np.log(test_data[numerical_feats] + 1)

class_weights = class_weight.compute_class_weight('balanced', classes=train_y.unique(), y=train_y)

#print('Numeric features: ', numerical_feats, '\nCategorical features:', categorical_feats, '\nOneHot features:', onehot_feats)

CPU times: user 1 s, sys: 163 ms, total: 1.16 s
Wall time: 1.23 s


## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

### Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
- The categorical features ['OS', 'Bsr', 'Rgn', 'TfcTp', 'VstTp', 'Wkd', 'Mo'] were both one hot encoded and category values re-encoded to be 0-indexed. This is due to the dual branch model architecture. The one hot encode categorical features are for the linear portion of our model to treat them all as independent discreet items. The re-encoding to be 0-indexed allows us to learn an embedding for each unique categorical value that attempts to encode the latent features of that category (similar to the goals of the SVD in the last competition). Those latent embedding features are then used as input to a DNN.
- For the numerical features, the ones that are not categorical, we applied a log on the feature values + 1 since we noticed that they all have a positive skew.
- Feature Engineering – Combined features and increased the score significantly. Specifically, we computed the average duration per page by calculating AdmDur/Adm, InfDur/Inf and PrdDur/Prd since we believed that the average time spent on a page per visit, rather than the total time across all visits might be a useful signal.

### 1. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)
- One hot encode categorical features: for the features ['OS', 'Bsr', 'Rgn', 'TfcTp', 'VstTp', 'Wkd', 'Mo'], we applied one hot encoding so that the model can interpret these as categories are not ordinal features. While the original features were numerical, we did not want the model to interpret these as ordered values since for many of these, they are unordered.
- The 0-indexed versions of the categorical features are used to choose the corresponding latent embedding from the matrix of size `(n_categoricals, n_features)` where `n_categoricals` is the number of unique elements in a given category and `n_features` is a parameter we can tune. After transforming we just pull the embedding from the given index corresponding to the encoded feature number.
- For the numerical features, the ones that are not categorical, we applied a log on the feature values + 1 since we noticed that they all have a positive skew. Applying the log helps to shift the values to have a more symmetrical distribution which can help the model learn patterns more easily.

### 2. How do you evaluate the effectiveness of these elements?
- The model was evaluated with stratified k fold cross validation using 5 folds. We decided on which feature engineering steps to include based on which ones helped improve the accuracy score on the validation sets.

### 3 What else have you tried that worked or didn't?
- We noticed that for Adm, Inf and Prd, for many observations, these values were non-zero i.e. there were visits on these pages but the corresponding durations AdmDur, InfDur and PrdDur were zero which seemed like a potential data logging issue. So for the observations where there were >0 visits but 0 duration, we made the visits zero as well. However in testing this transformation, it did not lead to any dramatic improvements in performance.

- We noticed that for the numerical features ['Adm', 'AdmDur', 'Inf', 'InfDur', 'PrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay', 'Prd'] many of these looked like they contained outliers in the long tail. Therefore we investigated approaches to remove outliers by evaluating the distributions and plots (histogram, boxplots). However, we noticed that the Kaggle test set exhibited very similar extreme values. This led us to conclude that these may not be outliers afterall and we should include these values so that the model could learn from these rare instances.
- From looking at the pairplots between features, we noticed that the classes were clearly separable using the PgVal features. It was unclear from this plot how much the other features would be helpful in this prediction problem. Therefore we hypothesized that there might be some features that are redundant and we could potentially drop. To analyse this, we conducted a feature ablation study where we would drop a feature and then re-run the model to observe how doing so would impact the accuracy score. We would repeat this for every feature. The model we tested on had an accuracy of 0.8706. Based on the results (shown below), we can see that every ablation led to a drop in accuracy, some more than others: PgVal and Wkd led to the biggest drops to 0.8324 and 0.8524 respectively. While others led to only small drops: both Adm and InfDur.
```
dropped to 0.8702.
# Drop Adm: 0.8702
# Drop AdmDur: 0.8694
# Drop Inf: 0.8659 *
# Drop InfDur: 0.8702
# Drop Prd: 0.8701
# Drop PrdDur: 0.8697
# Drop BncRt: 0.8678
# Drop ExtRt: 0.8695
# Drop PgVal: 0.8324 **
# Drop SpclDay: 0.8665
# Drop Mo: 0.8692
# Drop OS: 0.8721
# Drop Bsr: 0.8667
# Drop Rgn: 0.8648 *
# Drop TfcTp: 0.8686
# Drop VstTp: 0.8676
# Drop Wkd: 0.8524 **
```



## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

### Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.
- The model is based on the architecture from “Wide and Deep Learning” [2]. We build a neural network with two branches. One branch is a shallow linear model and the other a deep neural network, each branch taking slightly different data. For the wide branch we use as inputs all of the numerical features as well as the one hot encoding for the categorical features. For the deep branch we take the numerical features and the `LabelEncoded` categorical features.

  - **Deep**: The deep neural network is a three layer multi-layer perceptron with ReLU activation functions and BatchNormalization [3] between the layers. We take the same approach as [2] and decrease the number of units in each successive layer. The first layer has 1024 units, 512 in the second, and finally the last layer has 128.
  - **Wide**: The wide portion of the network is just a simple logistic classifier using the input features as well as the 128 output features from the deep branch bringing the total inputs to __.
- Additionally, the model was trained with a weighted loss function to combat the class imbalance. We weighted the majority class by 0.64 and the minority class by 2.23. This resulted in our predicted distributions matching the training distributions perfectly. 78% of our predictions are of class 0 and the remaining 22% of from class 1.
- One challenge with this model was the long runtime using the default optimization parameters. In order to achieve the desired 60 seconds run time we had to increase the batch size significantly but as outlined in that paper that typically comes at the cost of generalization performance.  We followed the advice in “Train longer, generalize better: close the generalization gap in large batch training of neural networks” by Hoffer et al [3] and “Accurate, large minibatch sgd: Training imagenet in 1 hour.” Goyal et al. [4]. First we increase the learning rate by the proposed function in [3]. Next, we decay the learning rate twice towards the end of training (rather than doing the first near the middle as is typically done) also recommended in [3]. For example, if we train for 35 epochs, we decay the learning rate at 25 and 30 epochs. This allowed us to train a relatively complex model within the time limit without sacrificing too much accuracy.


### 1. How did these decisions guide you in modeling?
- These decisions were guided by increases in model performance based on score as well as being able to keep within the time constraints.
- Wide and Deep networks was a topic of the modules reading and was very interesting to the group -- this interest guided us to evaluate its instead of a straightforward neural network.


### 2. How do you evaluate the effectiveness of these elements?
- The model was evaluated with stratified k fold cross validation using 5 folds. We decided that stratification was important due to the imbalance labels. We decided on which model to submit based on which model got the best average validation accuracy score while staying within the time limit.
- The model was passsed to teammates over Microsoft teams with each gain in performance. This enabled teammates to conduct code reviews and validate the results. It also enableed them to pursue options for improevements that could be fed back in.

### 3. What else have you tried that worked or didn't?
- We tried various neural network architectures steadily increasing the complexity until we hit the architecture from [2]. We started with basic MLP using only numeric features, we then tested various sizes of that model with more units, layers, or both. Next we tested a variant of a deep and wide network similar to that in [1] (using one-hot encodings for both the wide and deep section, no embedding layers), again testing various sizes for the deep section there.
- We also tried various learning rates, batch sizes, and number of epochs but we were ultimately restricted by runtime for some combinations that achieved better cross validation scores.
Feature engineering was tried early and resulted in a score above the baseline.  As the DNN grew more sophisticated,  these additional features were abandoned as the focus was on refining and improving the model overall. After hyper-parameters were identified that optimized performance (and score), feature engineering was revisited and resulted in an increase in score.
- Specific performance issues with the model hanging after the *fit* was researched [7]. The following code was added:

```
keras.layers.BatchNormalization._USE_V2_BEHAVIOR = False
```


- This value, by default is *True* and impacts the Batch Normalization Layer; the statistics resulting from training are using during testing or inference. When this value is *False*, a running average is used. This seems to fix the issue in most cases.

- Additionally, `amsgrad=False` was added to the Adam optimizer. While this is the default, in testing, setting this proved to be beneficial in addressing issues with the model stalling after the fit.


## Wide and Deep NN

In [ ]:
%%time
def build_wnd_model(n_numeric, n_onehot, embed_dims=5):
  '''
  builds the model from Figure 4. in https://arxiv.org/abs/1606.07792
  '''

  # we'll need various inputs to track all the varous routing through the model
  input_numericals = keras.layers.Input(shape=n_numeric)
  input_onehots = keras.layers.Input(shape=n_onehot)

  # input for each categorical variable for their respective embedding
  input_os = keras.layers.Input(shape=(1,))
  input_bsr = keras.layers.Input(shape=(1,))
  input_rgn = keras.layers.Input(shape=(1,))
  input_tfctp = keras.layers.Input(shape=(1,))
  input_vsttp = keras.layers.Input(shape=(1,))
  input_wkd = keras.layers.Input(shape=(1,))
  input_mo = keras.layers.Input(shape=(1,))


  # embeddings for each categorical variable
  emb_os = keras.layers.Embedding(
      input_dim=train_x['OS'].nunique() + 1, output_dim=embed_dims)(input_os)
  emb_os = keras.layers.Flatten()(emb_os)

  emb_bsr = keras.layers.Embedding(
      input_dim=train_x['Bsr'].nunique() + 1, output_dim=embed_dims)(input_bsr)
  emb_bsr = keras.layers.Flatten()(emb_bsr)

  emb_rgn = keras.layers.Embedding(
      input_dim=train_x['Rgn'].nunique() + 1, output_dim=embed_dims)(input_rgn)
  emb_rgn = keras.layers.Flatten()(emb_rgn)

  emb_tfctp = keras.layers.Embedding(
      input_dim=train_x['TfcTp'].nunique() + 1, output_dim=embed_dims)(input_tfctp)
  emb_tfctp = keras.layers.Flatten()(emb_tfctp)

  emb_vsttp = keras.layers.Embedding(
      input_dim=train_x['VstTp'].nunique() + 1, output_dim=embed_dims)(input_vsttp)
  emb_vsttp = keras.layers.Flatten()(emb_vsttp)

  emb_wkd = keras.layers.Embedding(
      input_dim=train_x['Wkd'].nunique() + 1, output_dim=embed_dims)(input_wkd)
  emb_wkd = keras.layers.Flatten()(emb_wkd)

  emb_mo = keras.layers.Embedding(
      input_dim=train_x['Mo'].nunique() + 1, output_dim=embed_dims)(input_mo)
  emb_mo = keras.layers.Flatten()(emb_mo)

  # the deep part uses the numerical data and the categorical embeddings
  input_deep = keras.layers.Concatenate()([input_numericals, emb_os, emb_bsr, emb_rgn, emb_tfctp, emb_vsttp, emb_wkd, emb_mo])

  # three layer deep network
  bn_in = keras.layers.BatchNormalization()(input_deep)
  hidden_a = keras.layers.Dense(
      512,
      activation="relu",
      kernel_initializer="he_normal",
      #kernel_regularizer=tf.keras.regularizers.L2(l2=1e-6),
      name='hidden1_a'
  )(bn_in)
  bn_a = keras.layers.BatchNormalization()(hidden_a)
  hidden_b = keras.layers.Dense(
      256,
      activation="relu",
      kernel_initializer="he_normal",
      #kernel_regularizer=tf.keras.regularizers.L2(l2=1e-6),
      name='hidden_b'
  )(bn_a)
  bn_b = keras.layers.BatchNormalization()(hidden_b)
  hidden_c = keras.layers.Dense(
      128,
      activation="relu",
      kernel_initializer="he_normal",
      #kernel_regularizer=tf.keras.regularizers.L2(l2=1e-6),
      name='hidden_c'
  )(hidden_b)
  bn_c = keras.layers.BatchNormalization()(hidden_c)

  # the wide part is the numericals and the onehots, combine with the deep output
  concat = keras.layers.Concatenate()([input_numericals, input_onehots, bn_c])

  # output pools all the above into a single prediction
  output = keras.layers.Dense(1, activation="sigmoid")(concat)

  m = keras.Model(
      inputs=[input_numericals, input_onehots, input_os, input_bsr, input_rgn, input_tfctp, input_vsttp, input_wkd, input_mo], outputs=[output])

  return m


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs


In [ ]:
%%time
tf.random.set_seed(0)   # always seed your experiments

BS = 4096 * 3
EPOCHS = 35
base_lr = 0.002

LR = np.sqrt(BS / 32) * base_lr # from https://arxiv.org/abs/1705.08741


def lr_scheduler(epoch, lr):
  # learning rate warmup: discussion paper uses gradient noise during this
  # stage but I can't do that with keras so we borrow from https://arxiv.org/abs/1706.02677
  # and do a learning rate warmup instead
  if epoch == 0:
    lr *= 0.1
  # back to default
  elif epoch == 1:
    lr *= 10

  if epoch in [25, 30]:
    lr *= 0.1

  return lr

lr_sched = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
es = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

CPU times: user 53 ms, sys: 0 ns, total: 53 ms
Wall time: 71.7 ms


In [ ]:
'''
%%time
kf = StratifiedKFold(n_splits=5, random_state = 0,shuffle=True)
kf.get_n_splits(train_x, train_y)

split_performance = []

for i, (train_index, test_index) in enumerate(kf.split(train_x, train_y)):
  # we don't want the preprocessing to hold over between runs
  _train_x, _train_y = train_x.copy(), train_y.copy()

  # start a fresh model for each fold
  m = build_wnd_model(len(numerical_feats), len(onehot_feats))

  m.compile(
      loss="binary_crossentropy",
      optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
      metrics=['accuracy']
  )

  #scaler = StandardScaler()
  #_train_x.iloc[train_index][numerical_feats] = scaler.fit_transform(_train_x.iloc[train_index][numerical_feats])
  #_train_x.iloc[test_index][numerical_feats] = scaler.transform(_train_x.iloc[test_index][numerical_feats])

  hist = m.fit(
    [_train_x.iloc[train_index][numerical_feats],
     _train_x.iloc[train_index][onehot_feats]
    ] + [_train_x.iloc[train_index][cat_feat] for cat_feat in categorical_feats],
    _train_y.iloc[train_index],
    epochs=EPOCHS,
    validation_data=(([
        _train_x.iloc[test_index][numerical_feats],
        _train_x.iloc[test_index][onehot_feats]
      ] + [_train_x.iloc[test_index][cat_feat] for cat_feat in categorical_feats], _train_y.iloc[test_index])),
    batch_size=BS,
    class_weight={i: class_weights[i] for i in range(2)},
    callbacks=[lr_sched],
    verbose=False,
  )
  split_performance.append(hist.history['val_accuracy'][-1])

print('Mean Val Acc:', np.mean(split_performance))

'''

'\n%%time\nkf = StratifiedKFold(n_splits=5, random_state = 0,shuffle=True)\nkf.get_n_splits(train_x, train_y)\n\nsplit_performance = []\n\nfor i, (train_index, test_index) in enumerate(kf.split(train_x, train_y)):\n  # we don\'t want the preprocessing to hold over between runs\n  _train_x, _train_y = train_x.copy(), train_y.copy()\n\n  # start a fresh model for each fold\n  m = build_wnd_model(len(numerical_feats), len(onehot_feats))\n\n  m.compile(\n      loss="binary_crossentropy",\n      optimizer=tf.keras.optimizers.Adam(learning_rate=LR),\n      metrics=[\'accuracy\']\n  )\n\n  #scaler = StandardScaler()\n  #_train_x.iloc[train_index][numerical_feats] = scaler.fit_transform(_train_x.iloc[train_index][numerical_feats])\n  #_train_x.iloc[test_index][numerical_feats] = scaler.transform(_train_x.iloc[test_index][numerical_feats])\n\n  hist = m.fit(\n    [_train_x.iloc[train_index][numerical_feats],\n     _train_x.iloc[train_index][onehot_feats]\n    ] + [_train_x.iloc[train_index][cat

# Make Prediction

In [ ]:
%%time
keras.layers.BatchNormalization._USE_V2_BEHAVIOR = False
#scaler = StandardScaler()
#train_x[numerical_feats] = scaler.fit_transform(train_x[numerical_feats])
#test_data[numerical_feats] = scaler.transform(test_data[numerical_feats])

m = build_wnd_model(
    len(numerical_feats),
    len(onehot_feats)
)

m.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR,amsgrad=False),
    metrics=['accuracy']
)

m.fit(
  [train_x[numerical_feats],
   train_x[onehot_feats]
  ] + [train_x[cat_feat] for cat_feat in categorical_feats],
  train_y,
  epochs=EPOCHS,
  batch_size=BS,
  class_weight={i: class_weights[i] for i in range(2)},
  callbacks=[lr_sched]
)

Epoch 1/35
37/37 [==============================] - 5s 23ms/step - loss: 0.5189 - accuracy: 0.7292 - lr: 0.0039
Epoch 2/35
37/37 [==============================] - 1s 20ms/step - loss: 0.4803 - accuracy: 0.7523 - lr: 0.0392
Epoch 3/35
37/37 [==============================] - 1s 20ms/step - loss: 0.4051 - accuracy: 0.8084 - lr: 0.0392
Epoch 4/35
37/37 [==============================] - 1s 25ms/step - loss: 0.3389 - accuracy: 0.8537 - lr: 0.0392
Epoch 5/35
37/37 [==============================] - 1s 27ms/step - loss: 0.2820 - accuracy: 0.8877 - lr: 0.0392
Epoch 6/35
37/37 [==============================] - 1s 28ms/step - loss: 0.2280 - accuracy: 0.9147 - lr: 0.0392
Epoch 7/35
37/37 [==============================] - 1s 29ms/step - loss: 0.1884 - accuracy: 0.9318 - lr: 0.0392
Epoch 8/35
37/37 [==============================] - 1s 32ms/step - loss: 0.1572 - accuracy: 0.9453 - lr: 0.0392
Epoch 9/35
37/37 [==============================] - 1s 36ms/step - loss: 0.1382 - accuracy: 0.9528 - lr:

In [ ]:
%%time
pY = pd.DataFrame(m.predict([test_data[numerical_feats], test_data[onehot_feats]] + [test_data[cat_feat] for cat_feat in categorical_feats]), index=test_data.index+1, columns=['Rev'])
ToCSV(pY.round(0).astype(int), 'wide-n-deep')
train_y.value_counts()/len(train_y)                # distribution of training target level
pY.round(0).astype(int).value_counts()/len(pY)     # distribution of test target level

1563/1563 [==============================] - 4s 2ms/step
CPU times: user 5.37 s, sys: 252 ms, total: 5.62 s
Wall time: 4.68 s


Rev
0      0.78
1      0.22
dtype: float64

# **References:**

[1] Géron, A. (2019). Hands-on machine learning with Scikit-learn, Keras, and Tensorflow: Concepts, tools, and techniques to build intelligent systems (2nd edition). O'Reilly Media.

[2] Cheng, H. T., Koc, L., Harmsen, J., Shaked, T., Chandra, T., Aradhye, H., ... & Shah, H. (2016, September). Wide & deep learning for recommender systems. In Proceedings of the 1st workshop on deep learning for recommender systems (pp. 7-10).

[3] Ioffe, S., & Szegedy, C. (2015, June). Batch normalization: Accelerating deep network training by reducing internal covariate shift. In International conference on machine learning (pp. 448-456). pmlr.

[4] Hoffer, E., Hubara, I., & Soudry, D. (2017). Train longer, generalize better: closing the generalization gap in large batch training of neural networks. Advances in neural information processing systems, 30.

[5] Goyal, P., Dollár, P., Girshick, R., Noordhuis, P., Wesolowski, L., Kyrola, A., ... & He, K. (2017). Accurate, large minibatch sgd: Training imagenet in 1 hour. arXiv preprint arXiv:1706.02677.

[6] Scikit-learn.org, 'API Reference — scikit-learn 1.2.2 documentation.' [Online]. Available: https://scikit-learn.org/stable/modules/classes.html

[7] Tensorflow. (n.d.). Hangs on model.fit · issue #36072 · Tensorflow/Tensorflow. GitHub. https://github.com/tensorflow/tensorflow/issues/36072

[8] Tf.keras.optimizers.Adam  :  tensorflow V2.13.0. TensorFlow. (n.d.). https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 50 sec


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)